<a href="https://colab.research.google.com/github/hammad93/hurricane-tts/blob/main/hurricane_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation
Please make sure to run all tests.

In [ ]:
!git clone https://github.com/hammad93/hurricane-tts.git

Cloning into 'hurricane-tts'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 105 (delta 57), reused 65 (delta 27), pack-reused 0
Receiving objects: 100% (105/105), 1.14 MiB | 2.47 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [ ]:
!pip install -r hurricane-tts/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import os
if not os.getenv("AZURE_OPENAI_API_KEY") :
  from google.colab import userdata
  os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
if not os.getenv("AZURE_REDIS_KEY") :
  os.environ["AZURE_REDIS_KEY"] = userdata.get('AZURE_REDIS_KEY')

In [ ]:
%cd /content/hurricane-tts
!python test.py

/content/hurricane-tts
.Here are the constructed messages: [{'role': 'system', 'content': 'You are an AI assistant that helps people find information.'}, {'role': 'user', 'content': 'test'}]
..
----------------------------------------------------------------------
Ran 3 tests in 7.620s

OK


In [ ]:
import prompts
import utils
import db
# we generate prompts by ingesting live hurricane data and supported languages
storm_data = utils.transform_storm_data()
prompt_data = prompts.generate_prompts()

# metadata related to multilingual generation
supported_langs = prompts.unique_lang_list()
supported_langs_df = utils.tts_langs()

In [ ]:
# example
print(prompt_data['storms'][0])

These are the input storm records and forecasts.
Each record has a lat and lon according to their geographic coordinates and wind speed in knots.
Please respond with the 3 most relevant languages exactly as they appear from the supported language list other than English each delimited by a comma.
[{'type': 'history', 'lat': 5.1, 'lon': 149.0, 'time': '2023-11-09 18:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 5.3, 'lon': 148.4, 'time': '2023-11-10 00:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 5.5, 'lon': 147.9, 'time': '2023-11-10 06:00:00', 'wind_speed': 15}, {'type': 'forecast', 'lat': 5.9, 'lon': 147.4, 'time': '2023-11-10T18:00:00', 'wind_speed': 20}, {'type': 'forecast', 'lat': 6.4, 'lon': 146.5, 'time': '2023-11-11T06:00:00', 'wind_speed': 20}, {'type': 'forecast', 'lat': 6.8, 'lon': 145.3, 'time': '2023-11-11T18:00:00', 'wind_speed': 20}, {'type': 'forecast', 'lat': 7.1, 'lon': 143.9, 'time': '2023-11-12T06:00:00', 'wind_speed': 25}, {'type': 'forecast', 'lat

# Language Geography inference from tropical storm geographical coorindates
We're trying to answer the question, "Which languages do we produce speech for to report on this tropical storm?" This pipeline will produce an English output but also utilize massively multilingual capabilities such that emergency notices are also in local languages

In [ ]:
retries = 5 # sometimes it fails, so we retry it
storm_langs = {} # keys are storm id's and values are the languages
storm_chats = {} # stores chat histories for storms
for index, storm_prompt in enumerate(prompt_data['storms']) :
  storm_id = list(storm_data.keys())[index]
  # Get the languages to generate the report for this storm
  while retries > 0 :
    response = prompts.chat(system=prompt_data['system'], message=storm_prompt)
    print(response)
    result = utils.llm_response_transform(
        resp=response, supported_langs=supported_langs)
    print(result)
    if result :
      break
    else :
      retries = retries - 1
      print(f"Failed. Retries left: {retries}")
  if retries < 1 :
    raise Exception("Couldn't produce a correct output from LLM.")

  # store results
  storm_langs[storm_id] = {'names': ['English'] + result}
  storm_chats[storm_id] = {
      'history' : [{'role': 'system', 'content': prompt_data['system']},
                   {'role': 'user', 'content': storm_prompt},
                   {'role': 'assistant', 'content': response}]
  }

Here are the constructed messages: [{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nI have a system for emergency notification of tropical storms that utilizes official data sources and creates speech audio from text with a massively multilingual model.\nThis is the list of supported languages,\n{'Zapotec, Mitla', 'Kanuri, Manga', 'Tamil', 'Buamu', 'Lauje', 'Hebrew', 'Wa, Parauk', 'Oroko', 'Tsakhur', 'Chiquitano', 'Kuman', 'Rawang', 'Kera', 'Telugu', 'Kuwaataay', 'Mokole', 'Ifugao, Tuwali', 'Gapapaiwa', 'Tagbanwa, Calamian', 'Lolopo', 'Arop-Lokep', 'Yamba', 'Tol', 'Kupsapiiny', 'Kyaka', 'Mixtec, PeÃ±oles', 'Quichua, Napo', 'Maya, MopÃ¡n', 'Dendi', 'Ramoaaina', 'Wolio', 'Ukrainian', 'Balochi, Southern', 'Nanai', 'Blaan, Koronadal', 'Benga', 'Kriol', 'Sougb', 'Bashkort', 'Chin, Hakha', 'Kogi', 'MÃ\\xadskito', 'Quechua, Cajamarca', 'Tektiteko', 'Khmer, Northern', 'Yao', 'Tawbuid', 'Cerma', 'Daasanach', 'Nyankore', 'Waimaha', 'Icelan

# Tropical Storm Report
This code will create the report in the language specified based on the tropical storm.

In [ ]:
with open('prompts/report-prompt.txt', 'r') as file:
  report_prompt = file.read()
for storm in storm_langs:
  reports = []
  for lang in storm_langs[storm]['names']:
    print(storm_chats[storm]['history'])
    # Construct the prompt
    message = report_prompt.format(lang=lang)
    print(message)
    # Send to LLM
    response = prompts.chat(
        message = message, history = storm_chats[storm]['history'])
    print(response)
    # store data
    reports.append(response)
  storm_langs[storm]['reports'] = reports

[{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nI have a system for emergency notification of tropical storms that utilizes official data sources and creates speech audio from text with a massively multilingual model.\nThis is the list of supported languages,\n{'Zapotec, Mitla', 'Kanuri, Manga', 'Tamil', 'Buamu', 'Lauje', 'Hebrew', 'Wa, Parauk', 'Oroko', 'Tsakhur', 'Chiquitano', 'Kuman', 'Rawang', 'Kera', 'Telugu', 'Kuwaataay', 'Mokole', 'Ifugao, Tuwali', 'Gapapaiwa', 'Tagbanwa, Calamian', 'Lolopo', 'Arop-Lokep', 'Yamba', 'Tol', 'Kupsapiiny', 'Kyaka', 'Mixtec, PeÃ±oles', 'Quichua, Napo', 'Maya, MopÃ¡n', 'Dendi', 'Ramoaaina', 'Wolio', 'Ukrainian', 'Balochi, Southern', 'Nanai', 'Blaan, Koronadal', 'Benga', 'Kriol', 'Sougb', 'Bashkort', 'Chin, Hakha', 'Kogi', 'MÃ\\xadskito', 'Quechua, Cajamarca', 'Tektiteko', 'Khmer, Northern', 'Yao', 'Tawbuid', 'Cerma', 'Daasanach', 'Nyankore', 'Waimaha', 'Icelandic', 'Dinka, Southwestern', 'Maka'

In [ ]:
storm_langs

{'WP952023': {'names': ['English', 'Tagalog', 'Spanish', 'Swahili'],
  'reports': ['Here is the weather report:\n\nGood day everyone, this is your local weather report. A tropical storm is expected to bring heavy rain and strong winds to the area. As of now, the storm is located at latitude 5.1 and longitude 149.0 with wind speeds of 15 knots. It is expected to continue moving towards the West Pacific Ocean. \n\nHistorical data shows that the storm will move towards the West at similar wind speeds as it reaches latitudes 5.3, 5.5, and 5.9 with minimal changes to its wind speeds. \n\nFurthermore, forecasts predict that the storm will continue to move towards the West and weaken slightly as it reaches latitudes 6.4 and 6.8 with wind speeds of 20 knots. Upon reaching latitude 7.1, the storm is predicted to increase in wind speeds up to 25 knots. \n\nFinally, at the latitude of 7.5, the storm is expected to have wind speeds at 30 knots. \n\nPlease take precautions such as securing any loos

The following are from the TTS implementation
https://github.com/facebookresearch/fairseq/blob/main/examples/mms/tts/tutorial/MMS_TTS_Inference_Colab.ipynb


In [ ]:
%cd ~/
!git clone https://github.com/jaywalnut310/vits.git
%cd vits/

!pip install Cython==0.29.21
!pip install librosa==0.8.0
!pip install phonemizer==2.2.1
!pip install scipy
!pip install numpy
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install Unidecode==1.1.1

%cd monotonic_align/
%mkdir monotonic_align
!python3 setup.py build_ext --inplace
%cd ../
%pwd

/root
Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Receiving objects: 100% (81/81), 3.33 MiB | 5.76 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/root/vits
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.5
    Uninstalling Cython-3.0.5:
      Successfully uninstalled Cython-3.0.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.9 MB/s eta 0:00:00
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201367 sha256=b61d7fc072c3ef509cc5472b568ce11f078a7aa0bea7cc3dfefb458933955c4a
  Stored in directory: /root/.cache/pip/wheels/bf/b7/85/2f8044306ccec014930aea23ad4852fca9e2584e21c6972bc6
Successfully built librosa
  Attempting uninstall: librosa
  

'/root/vits'

In [ ]:
# we have the data we need and the utils class is also used in the next code
import sys
del sys.modules["utils"]

In [ ]:
%cd /content/hurricane-tts/
import fair_mms_tts
%cd /root/vits
import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn

/content/hurricane-tts
/root/vits


In [ ]:
# match language names to iso code
for storm in storm_langs :
  langs = storm_langs[storm]['names']
  print(langs)
  codes = [supported_langs_df[
      supported_langs_df['Language Name'] == name
  ]['ISO 693-3'].iloc[0] for name in langs]
  print(codes)
  storm_langs[storm]['codes'] = codes

['English', 'Tagalog', 'Spanish', 'Swahili']
['eng', 'tgl', 'spa', 'swh']
['English', 'Malay', 'Samoan', 'Bislama']
['eng', 'zlm', 'smo', 'bis']


In [ ]:
import soundfile as sf
def generate_audio(report, lang_code, out):
  '''
  Generates the audio given the report and lang_code
  '''
  ckpt_dir = fair_mms_tts.download(lang_code)
  import torch
  if torch.cuda.is_available():
      device = torch.device("cuda")
  else:
      device = torch.device("cpu")
  print(device)
  print(f"Run inference with {device}")
  vocab_file = f"{ckpt_dir}/vocab.txt"
  config_file = f"{ckpt_dir}/config.json"
  assert os.path.isfile(config_file), f"{config_file} doesn't exist"
  hps = utils.get_hparams_from_file(config_file)

  text_mapper = fair_mms_tts.TextMapper(vocab_file)
  net_g = SynthesizerTrn(
      len(text_mapper.symbols),
      hps.data.filter_length // 2 + 1,
      hps.train.segment_size // hps.data.hop_length,
      **hps.model)
  net_g.to(device)
  _ = net_g.eval()

  g_pth = f"{ckpt_dir}/G_100000.pth"
  print(f"load {g_pth}")

  _ = utils.load_checkpoint(g_pth, net_g, None)
  txt = fair_mms_tts.preprocess_text(report, text_mapper, hps, lang=lang_code)
  stn_tst = text_mapper.get_text(txt, hps)

  with torch.no_grad():
      x_tst = stn_tst.unsqueeze(0).to(device)
      x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
      hyp = net_g.infer(
          x_tst, x_tst_lengths, noise_scale=.667,
          noise_scale_w=0.8, length_scale=1.0
      )[0][0,0].cpu().float().numpy()

  print(f"Generated audio")
  sf.write(out, hyp, hps.data.sampling_rate)

In [ ]:
# generate audio
import time
time_fname = int(time.time())
outputs = []
for storm in storm_langs :
  current = storm_langs[storm]
  for index, code in enumerate(current['codes']):
    path = f"/content/{storm}_{time_fname}_{current['names'][index]}.wav"
    outputs.append(path)
    generate_audio(report = current['reports'][index],
                   lang_code = code,
                   out = path)

Download model for language: eng
Model checkpoints in ./eng: ['vocab.txt', 'config.json', 'G_100000.pth']
cuda
Run inference with cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


load ./eng/G_100000.pth
eng
text after filtering OOV: here is the weather reportgood day everyone this is your local weather report a tropical storm is expected to bring heavy rain and strong winds to the area as of now the storm is located at latitude 51 and longitude 140 with wind speeds of 15 knots it is expected to continue moving towards the west pacific ocean historical data shows that the storm will move towards the west at similar wind speeds as it reaches latitudes 53 55 and 5 with minimal changes to its wind speeds furthermore forecasts predict that the storm will continue to move towards the west and weaken slightly as it reaches latitudes 64 and 6 with wind speeds of 20 knots upon reaching latitude 1 the storm is predicted to increase in wind speeds up to 25 knots finally at the latitude of 5 the storm is expected to have wind speeds at 30 knots please take precautions such as securing any loose objects and avoiding travel if possible we will continue to provide updates as 

In [ ]:
import db
import json
print(outputs)
db.upload_latest_audios(outputs)

['/content/WP952023_1699666627_English.wav', '/content/WP952023_1699666627_Tagalog.wav', '/content/WP952023_1699666627_Spanish.wav', '/content/WP952023_1699666627_Swahili.wav', '/content/SH912023_1699666627_English.wav', '/content/SH912023_1699666627_Malay.wav', '/content/SH912023_1699666627_Samoan.wav', '/content/SH912023_1699666627_Bislama.wav']


In [ ]:
!git -C /content/hurricane-tts/ pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), 284 bytes | 284.00 KiB/s, done.
From https://github.com/hammad93/hurricane-tts
   6cfadee..e6d3d67  main       -> origin/main
Updating 6cfadee..e6d3d67
error: Your local changes to the following files would be overwritten by merge:
	db.py
Please commit your changes or stash them before you merge.
Aborting
